In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA med")
spark = SparkSession.builder.getOrCreate()

In [0]:
# load doc_chunks
chunks_df = spark.table("workspace.med.doc_chunks")
chunks_df = chunks_df.filter(F.col("chunk_text").isNotNull() & (F.length("chunk_text") > 0))